In [20]:
from data_rnn import load_ndfa, load_brackets
from data_prep import pad_and_convert
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

In [2]:
x_train_ndfa, (i2w_ndfa, w2i_ndfa) = load_ndfa(n=150_000)
x_train_brackets, (i2w_brackets, w2i_brackets) = load_brackets(n=150_000)

In [3]:
print(''.join([i2w_ndfa[i] for i in  x_train_ndfa[10_000]]) )

ss


In [4]:
print(''.join([i2w_brackets[i] for i in  x_train_brackets[10_000]]) )

()


In [31]:
len(x_train_ndfa[-1])

158

In [12]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, emb_size, h, num_char, n_layers=1):
        super(LSTM, self).__init__()

        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.lstm = nn.LSTM(input_size=emb_size, hidden_size=h, num_layers=n_layers, batch_first=True)
        self.fc = nn.Linear(h, num_char)
    
    def forward(self, input_seq, h):

        embedded = self.embedding(input_seq)
        lstm_out, hidden = self.lstm(embedded, h)
        # lstm_out = lstm_out[:, -1, :]
        output = self.fc(lstm_out)
        return output, hidden


In [ ]:
vocab_size = len(set(w2i_ndfa))
emb_size = 300
h = 300
num_char = vocab_size
n_layers = 1

In [24]:
model = LSTM(vocab_size=vocab_size, emb_size=emb_size, h=h, num_char=num_char, n_layers=1)

In [6]:
x_train_ndfa = pad_and_convert(x_train_ndfa)
x_train_brackets = pad_and_convert(x_train_brackets)

In [18]:
num_epochs = 1
learning_rate = 0.001

In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [8]:
x_train_ndfa.size()

torch.Size([150000, 158])

In [10]:
x_train_ndfa[1].size

<function Tensor.size>

In [21]:
dataset = TensorDataset(x_train_ndfa)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [26]:
epochs = 1
for epoch in range(epochs):
    total_loss = 0.0
    model.train()

    for batch, inputs in enumerate(dataloader):
        optimizer.zero_grad()

        output = model(batch)

        loss = criterion(output.view(-1, vocab_size), inputs.view(-1))

        loss.backward()

        optimizer.step()
        
        total_loss += loss.item()
    average_loss = total_loss / len(dataloader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss:.4f}')




TypeError: LSTM.forward() missing 1 required positional argument: 'h'